In [ ]:
import os
import requests
import gzip
import shutil
from typing import Optional, Union, Tuple, List
from dataclasses import dataclass, field

import causallift
from causallift import CausalLift

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

causallift.__version__
pd.options.display.max_rows = 8
seed = 42

In [ ]:
def prepare_data(data: str, **kwargs):

    if data == 'simulated_observational_data':
        """
        # Generate simulated data
        # "Sleeping dogs" (a.k.a. "do-not-disturb"; people who will "buy" if not 
        treated but will not "buy" if treated) can be simulated by negative values 
        in tau parameter.
        # Observational data which includes confounding can be simulated by 
        non-zero values in propensity_coef parameter.  
        # A/B Test (RCT) with a 50:50 split can be simulated by all-zeros values 
        in propensity_coef parameter (default).
        # The first element in each list parameter specifies the intercept.
        """
        from causallift import generate_data

        np.random.seed(seed)

        df = generate_data(
            N=20000, 
            n_features=10, 
            beta=np.random.uniform(low=-3.0, high=5.0, size=10+1).tolist(), # Effect of [intercept and features] on outcome 
            error_std=0.1, 
            tau=np.random.uniform(low=-1.0, high=1.0, size=10+1).tolist(), # Effect of [intercept and features] on treated outcome
            tau_std=0.1, 
            discrete_outcome=True, 
            seed=seed, 
            feature_effect=0, # Effect of beta on treated outcome
            propensity_coef=np.random.randint(low=-1, high=1, size=10+1).tolist(), # Effect of [intercept and features] on propensity log-odds for treatment
            index_name='index',
        )
        
    elif data == 'lalonde':
        r""" 
            Lalonde dataset was used to evaluate propensity score in the paper:
            Dehejia, R., & Wahba, S. (1999). Causal Effects in Nonexperimental 
            Studies: Reevaluating the Evaluation of Training Programs. Journal of 
            the American Statistical Association, 94(448), 1053-1062. 
            doi:10.2307/2669919

            Lalonde dataset is now included in R package named "Matching."
            http://sekhon.berkeley.edu/matching/lalonde.html
        """
        def get_lalonde():
            r""" Load datasets, concatenate, and create features to get data frame 
            similar to 'lalonde' that comes with "Matching.")
            """
            cols = ['treat', 'age', 'educ', 'black', 'hisp', 'married', 'nodegr','re74','re75','re78']
            control_df = pd.read_csv('http://www.nber.org/~rdehejia/data/nswre74_control.txt', sep=r'\s+', header = None, names = cols)
            treated_df = pd.read_csv('http://www.nber.org/~rdehejia/data/nswre74_treated.txt', sep=r'\s+', header = None, names = cols)
            lalonde_df = pd.concat([control_df, treated_df], ignore_index=True)
            lalonde_df['u74'] = np.where(lalonde_df['re74'] == 0, 1.0, 0.0)
            lalonde_df['u75'] = np.where(lalonde_df['re75'] == 0, 1.0, 0.0)
            return lalonde_df
        lalonde_df = get_lalonde()
        
        """ Prepare the input Data Frame. """
        df = lalonde_df.copy()
        df.rename(columns={'treat':'Treatment', 're78':'Outcome'}, inplace=True)
        df['Outcome'] = np.where(df['Outcome'] > 0, 1.0, 0.0)
        
        # categorize age to 20s, 30s, 40s, and 50s and then one-hot encode age
        df.loc[:,'age'] = df['age'].apply(lambda x:'{:.0f}'.format(x)[:-1]+'0s') 
        df = pd.get_dummies(df, columns=['age'], drop_first=True) 
        
        cols = ['nodegr', 'black', 'hisp', 'age_20s', 'age_30s', 'age_40s', 'age_50s', 
                'educ', 'married', 'u74', 'u75', 'Treatment', 'Outcome']
        df = df[cols]

    elif data == 'criteo':
        save_dir = "./raw_data"
        os.makedirs(save_dir, exist_ok=True)

        criteo_url = "http://go.criteo.net/criteo-research-uplift-v2.1.csv.gz"
        zip_file_name = "raw_data/criteo-research-uplift-v2.1.csv.gz"
        unzip_file_name = "raw_data/criteo-research-uplift-v2.1.csv"
        
        if os.path.isfile(unzip_file_name):
            print("The downloaded file already exists!")
        
        else:
            print("Try to download the raw data from the server...")
            response = requests.get(criteo_url, stream=True)
            total_size_in_bytes = int(response.headers.get("content-length", 0))
            block_size = 1024
            progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
            with open(zip_file_name, "wb") as f:
                for data in response.iter_content(block_size):
                    progress_bar.update(len(data))
                    f.write(data)
            progress_bar.close()
            print("Finished downloading!!!")
            if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
                print("Error, something went wrong!")
                return

            print("Try to unzip the downloaded file")
            with gzip.open(zip_file_name, "rb") as f_in:
                with open (unzip_file_name, "wb") as f_out:
                    shutil.copyfileobj(f_in, f_out)

            os.remove(zip_file_name)
            print("Zip file removed from disk")

        print("Import the csv file into pd.DataFrame")
        df = pd.read_csv(unzip_file_name)
    
    else:
        raise ValueError("No corresponding data found")

    return df

In [ ]:
df = prepare_data("simulated_observational_data")

In [ ]:
np.sum(df["Treatment"] == 1)/len(df)
# 15% is treated

In [ ]:
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)
print(len(train_df), len(eval_df))

In [ ]:
train_df.head()

In [ ]:
corr = train_df.corr()
corr

In [ ]:
plt.imshow(corr, cmap="bwr")
plt.colorbar()
plt.clim(-1, 1)
plt.show()

In [ ]:
pd.crosstab(train_df["Treatment"], train_df["Outcome"])/len(train_df)

In [ ]:
for idx, column in enumerate(df.columns):
    if idx >= len(df.columns) - 2:
        # t and y varaibles
        continue
    plt.hist(df[column], density=True, histtype="step", label=column)
plt.legend()

In [ ]:
class UpliftDataset(Dataset): 
    def __init__(
        self,
        df: pd.DataFrame, 
        in_features: int,
        t_idx: Optional[int] = None, 
        y_idx: Optional[int] = None,
    ):
        t_idx = in_features if t_idx is None else t_idx
        y_idx = in_features + 1 if y_idx is None else y_idx

        self.in_features = in_features
        self.t_idx = t_idx
        self.y_idx = y_idx

        self.df = df
        self.X = self.df.iloc[:, 0:in_features]
        self.t = self.df.iloc[:, t_idx]
        self.y = self.df.iloc[:, y_idx]

    def __len__(self): 
        return len(self.X)

    def __getitem__(self, idx):
        X = torch.tensor(self.X.iloc[idx, :].to_numpy(), dtype=torch.float32)
        t = torch.tensor(self.t.iloc[idx], dtype=torch.float32)
        y = torch.tensor(self.y.iloc[idx], dtype=torch.float32)
        return (X, t, y)

In [ ]:
num_features = 10

train_set = UpliftDataset(train_df, num_features)
eval_set  = UpliftDataset(eval_df, num_features)

train_dl = DataLoader(train_set, batch_size=128, shuffle=True)
eval_dl  = DataLoader(eval_set, batch_size=128, shuffle=False)

In [ ]:
train_set[0]

In [ ]:
class ComplexModel(nn.Module): 
    def __init__(
        self,
        in_features: int,
    ):
        super().__init__()
        self.expand = nn.Linear(in_features+1, 128)
        self.fc1 = nn.Linear(128, 128) 
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 128) 
        self.bn2 = nn.BatchNorm1d(128)
        self.classifier = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):

        x = self.expand(x)
        _x = torch.tanh(self.bn1(self.fc1(x))) 
        x = self.dropout(_x) + x
        _x = torch.tanh(self.bn2(self.fc2(x)))
        x = self.dropout(_x) + x
        x = self.classifier(x)
        return x

In [ ]:
train_set[0]
class UpliftWrapper(nn.Module): 
    def __init__(
        self,
        in_features: int,
    ):
        super(UpliftWrapper, self).__init__() 
        self.in_features = in_features 
        self.model = ComplexModel(in_features) 
        self.sigmoid = nn.Sigmoid()
        
    def forward(
        self, 
        x: torch.Tensor, 
        t: torch.Tensor,
    ):
        """Forward function for Uplift model
        Args:
            x: `torch.Tensor`
            t: `torch.Tensor`
        Returns:
            `dict[str, torch.Tensor]`
        """
        # X shape: (B, N)
        if t.ndim == 2:
            t = t.squeeze()
        B = x.size(0)
        L = x.size(1)
        # print(f"input shape: {x.shape}")
        
        # first creating the inputs accordingly
        x_0 = torch.cat([x, torch.zeros([B, 1]).to(x.device)], dim=1) 
        x_1 = torch.cat([x, torch.ones([B, 1]).to(x.device)], dim=1)

        y_0 = self.sigmoid(self.model(x_0)).squeeze()
        y_1 = self.sigmoid(self.model(x_1)).squeeze()
        
        pred = torch.where(t == 1, y_1, y_0)
        return {
            "uplift": y_1 - y_0, "pred": pred,
        }

In [ ]:
class DirectUpliftLoss(nn.Module): 
    def __init__(self,
        propensity_score: float = 0.5,
        alpha: Optional[float] = None, 
    ):
        super(DirectUpliftLoss, self).__init__()
        if alpha > 1 or alpha < 0:
            raise ValueError("alpha must be in [0, 1]")
        self.e_x = propensity_score 
        self.alpha = alpha

        self.loss_u = nn.MSELoss() 
        self.loss_y = nn.BCELoss()

    def forward(self, out, t, y):
        z = t * y / self.e_x - (1-t) * y / (1-self.e_x) 
        # variable transformation
        
        loss_uplift = self.loss_u(out["uplift"], z) 
        loss_pred = self.loss_y(out["pred"], y)
        loss = (1-self.alpha) * loss_uplift + self.alpha * loss_pred 
        
        return loss

In [ ]:
model = UpliftWrapper(10)
criterion = DirectUpliftLoss(0.5, 0.15)

In [ ]:
train_set[0][0].unsqueeze(0)

In [ ]:
model.eval()
out = model(train_set[0][0].unsqueeze(0), train_set[0][1])
out

In [ ]:
X, t, y = next(iter(train_dl))
out = model(X, t)
# print(out)
# print(out["uplift"].shape, out["pred"].shape)

In [ ]:
out["pred"].shape, out["uplift"].shape

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].hist(out["pred"].detach().cpu().numpy())
ax[1].hist(out["uplift"].detach().cpu().numpy())

In [ ]:
# loss = criterion(out, t, y)
loss = criterion(out, t, y)
loss

In [ ]:
z = t * y / 0.5 - (1-t) * y / (1-0.5)
z

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)

In [ ]:
model.cuda()

num_epochs = 100
train_losses = []
train_accuracies = []
eval_losses = []
eval_accuracies = []

for epoch in range(num_epochs):
    print(f"epoch: {epoch}")

    train_cnt = 0
    train_corrects = 0

    model.train()
    for X, t, y in tqdm(train_dl):
        optimizer.zero_grad()
        train_cnt += X.size(0)

        X = X.cuda()
        t = t.cuda()
        y = y.cuda()

        out = model(X, t)
        loss = criterion(out, t, y)
        # loss = criterion(out["pred"], y)
        loss.backward()
        train_losses.append(loss.item())

        optimizer.step()

        pred = np.where(out["pred"].detach().cpu().numpy() > 0.5, 1, 0)
        train_corrects += np.sum(pred == y.cpu().numpy())

    train_accuracies.append(train_corrects/train_cnt)
    print(f"train accuracy: {train_accuracies[-1]}")

    eval_cnt = 0
    eval_corrects = 0

    model.eval()
    with torch.no_grad():
        for X, t, y in tqdm(eval_dl):
            eval_cnt += X.size(0)

            X = X.cuda()
            t = t.cuda()
            y = y.cuda()

            out = model(X, t)
            loss = criterion(out, t, y)
            # loss = criterion(out["pred"], y)
            eval_losses.append(loss.item())

            pred = np.where(out["pred"].detach().cpu().numpy() > 0.5, 1, 0)
            eval_corrects += np.sum(pred == y.cpu().numpy())

    eval_accuracies.append(eval_corrects/eval_cnt)
    print(f"train accuracy: {eval_accuracies[-1]}")

    print()


In [ ]:
plt.hist(out["pred"].detach().cpu().numpy(), bins=np.arange(0, 1.1, 0.1))

In [ ]:
out["uplift"]

In [ ]:
plt.plot(train_losses)